In [55]:
import pandas as pd
import os
import plotly.express as px
import plotly.subplots as sp
import textwrap

In [56]:
def get_json_files(folder):
    return [f for f in os.listdir(folder) if f.endswith(".json")]

In [57]:
def calculate_accuracy(df):
    df[df['chatbot_response_clean'] == df['label']].shape[0]
    return df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]


In [58]:
def calculate_metrics(df):
    # Calculate true positives, false positives, false negatives
    df['true_positives'] = df.apply(lambda row: 1 if row['chatbot_response_clean'] == row['label'] and row['label'] == 1 else 0, axis=1)
    df['false_positives'] = df.apply(lambda row: 1 if row['chatbot_response_clean'] != row['label'] and row['label'] == 0 else 0, axis=1)
    df['false_negatives'] = df.apply(lambda row: 1 if row['chatbot_response_clean'] != row['label'] and row['label'] == 1 else 0, axis=1)
    # Calculate precision and recall
    precision = df['true_positives'].sum() / (df['true_positives'].sum() + df['false_positives'].sum())
    recall = df['true_positives'].sum() / (df['true_positives'].sum() + df['false_negatives'].sum())
    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

In [59]:
def calculate_results(files, folder, model_name):
    results_cg = []

    for file in files:
        df = pd.read_json(os.path.join(folder, file))
        precision, recall, f1 = calculate_metrics(df)
        accuracy = calculate_accuracy(df)
        # get the chatbot_question of the 0th row
        question = df.iloc[0]['chatbot_question']
        results_cg.append({
            "model": model_name,
            "question": question,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        })
    return results_cg

In [60]:
def print_promt(index, row, model):
    if model == "CHAT-GPT":
        question_text = ""
        
        for message in eval(row['question_label']):
            question_text += message.get("content", "")
    else:
        question_text = row['question_label']

        
    wrapped_text = textwrap.fill(question_text, width=120, subsequent_indent='    ')  # Wrap text at a maximum width of 60 characters
    print(f"Prompt for index {index}:\n    {wrapped_text}\n")

In [61]:
def create_chart(results):
    # plot the results
    df = pd.DataFrame(results)
    # Convert the 'question' column to a string representation for display
    df['question_label'] = df['question'].apply(str)
    # Convert the index to a string representation for display
    df['index_label'] = df.index.astype(str)

    # Create separate bar charts for accuracy and F1 score
    fig_acc = px.bar(df, x='index_label', y='accuracy', labels={'index_label': 'Index', 'accuracy': 'Accuracy'})
    fig_prec = px.bar(df, x='index_label', y='precision', labels={'index_label': 'Index', 'precision': 'Precision'})
    fig_recall = px.bar(df, x='index_label', y='recall', labels={'index_label': 'Index', 'recall': 'Recall'})
    fig_f1 = px.bar(df, x='index_label', y='f1', labels={'index_label': 'Index', 'f1': 'F1 Score'})

    # Combine the two charts into a single plot
    fig = sp.make_subplots(rows=2, cols=2, subplot_titles=(f'Precision {results[0].get("model")}', f'Recall {results[0].get("model")}', f'Accuracy {results[0].get("model")}', f'F1 Score {results[0].get("model")}'))
    fig.add_trace(fig_acc.data[0], row=2, col=1)
    fig.add_trace(fig_prec.data[0], row=1, col=1)
    fig.add_trace(fig_recall.data[0], row=1, col=2)
    fig.add_trace(fig_f1.data[0], row=2, col=2)



    # Customize the chart
    fig.update_layout(
        title=f'Results {results[0].get("model")}',
        xaxis_tickangle=-45,
        yaxis=dict(range=[0, 1]),  # Set the range for the first y-axis
        yaxis2=dict(range=[0, 1]),  # Set the range for the second y-axis
        yaxis3=dict(range=[0, 1]),  # Set the range for the third y-axis
        yaxis4=dict(range=[0, 1]),  # Set the range for the fourth y-axis
    )

    # Show the chart
    fig.show()

    # Print the question and their index
    for index, row in df.iterrows():
        print_promt(index, row, results[0].get("model"))

In [62]:
def create_models_chart(results, title):
    # plot the results
    df = pd.DataFrame(results)
    # Convert the 'question' column to a string representation for display
    df['question_label'] = df['question'].apply(lambda x: str(x))
    # Convert the index to a string representation for display
    df['index_label'] = df.index.astype(str)

    # Create separate bar charts for accuracy and F1 score
    fig_acc = px.bar(df, x='model', y='accuracy', labels={'model': 'Index', 'accuracy': 'Accuracy'})
    fig_prec = px.bar(df, x='model', y='precision', labels={'index_label': 'Index', 'precision': 'Precision'})
    fig_recall = px.bar(df, x='model', y='recall', labels={'index_label': 'Index', 'recall': 'Recall'})
    fig_f1 = px.bar(df, x='model', y='f1', labels={'index_label': 'Index', 'f1': 'F1 Score'})

    # Combine the two charts into a single plot
    fig = sp.make_subplots(rows=2, cols=2, subplot_titles=(f'Precision', f'Recall', f'Accuracy', f'F1 Score'))
    fig.add_trace(fig_acc.data[0], row=2, col=1)
    fig.add_trace(fig_prec.data[0], row=1, col=1)
    fig.add_trace(fig_recall.data[0], row=1, col=2)
    fig.add_trace(fig_f1.data[0], row=2, col=2)



    # Customize the chart
    fig.update_layout(
        title=title,
        xaxis_tickangle=-45,
        yaxis=dict(range=[0, 1]),  # Set the range for the first y-axis
        yaxis2=dict(range=[0, 1]),  # Set the range for the second y-axis
        yaxis3=dict(range=[0, 1]),  # Set the range for the third y-axis
        yaxis4=dict(range=[0, 1]),  # Set the range for the fourth y-axis
    )

    # Show the chart
    fig.show()

    # Print the question and their index
    for index, row in df.iterrows():
        print_promt(index, row, results[0].get("model"))

## Analyze ChatGPT3.5

### Random Sample v2

In [63]:
folder = "data/results/chat_gpt/new_random_sample"
files = get_json_files(folder)


results_cg = calculate_results(files, folder, model_name="CHAT-GPT")
create_chart(results_cg)

Prompt for index 0:
    Do the following two product decriptions match. Answer with yes or no. product 1: Maxxis Maxxis Minion DHR2 29 x 2.3
    Folding Bead, product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Folding Bead)



### Normal benchmark

In [64]:
folder = "data/results/chat_gpt/curated"
files = get_json_files(folder)


results_cg = calculate_results(files, folder, model_name="CHAT-GPT")
create_chart(results_cg)

Prompt for index 0:
    You are taking part in a benchmark that tries to measure your performance at entity matching. You should determine if
    two products are the exact same. Two products are only the same if all attributes match! Only answer yes or
    no.product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0 it costs: 9,86 EUR, product 2: SanDisk 64GB
    SDIX30N-064G-GN6NN USB 3.0 Flash Drive 35.85 USD

Prompt for index 1:
    You are taking part in a benchmark that tries to measure your performance at entity matching. You should determine if
    two products are the exact same. Two products are only the same if all attributes match! Only answer yes or
    no.product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash
    Drive

Prompt for index 2:
    You are a helpful assistant that tries to tell if two products are the same. Two products are only the same if all
    attributes match! Only answer yes or no.product 1: Pen Dri

### Hard benchmark

In [65]:
folder = "data/results/chat_gpt"
files = get_json_files(folder)

# exclude 
files = [f for f in files if f != "2023-04-24 16:08:16.804316_simple_promt.json"]

results_cg = calculate_results(files, folder, model_name="CHAT-GPT")
create_chart(results_cg)




Prompt for index 0:
    You are a helpful assistant that tries to tell if two products are the same. Two products are only the same if all
    attributes match! Only answer yes or no.product 1: Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided
    printing 250 sheet Network 46PPM, product 2: Brother HL-L6300DW Laser Printer - Monochrome - DuplexThe twoPlease
    answer yes or no otherwise I will not be able to understand you.product 1: Brother HL-L6200DW Wireless High Speed
    Mono Laser 2-sided printing 250 sheet Network 46PPM, product 2: Brother HL-L6300DW Laser Printer - Monochrome -
    DuplexNo.Very good, thank you!product 1: Corsair Vengeance LPX RAM 16GB 2400MHz DDR4 UDIMM 288 Pin Desktop Memory,
    product 2: 16GB Corsair Vengeance LPX schwarz DDR4-2400 DIMM CL14 SingleNo.Thats incorrect, they are the
    same!product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer
    Monaco Watch

Prompt for index 1:
    Are the

## Analyze ChatGPT4

In [66]:
folder = "data/results/chat_gpt_4"
files = get_json_files(folder)

# exclude 
files = [f for f in files if f != "2023-04-24 16:08:16.804316_simple_promt.json"]

results_cg = calculate_results(files, folder, model_name="CHAT-GPT4")
create_chart(results_cg)




Prompt for index 0:
    Do the following two product decriptions match. Answer with yes or no. product 1: Pen Drive SanDisk 64GB Cruzer Glide
    USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive



## Analyse GPT4All

### Random sample 1000

In [67]:
folder = "data/results/chat_gpt/random_sample_1000"
files = get_json_files(folder)

results_cg = calculate_results(files, folder, model_name="GPT4All")
create_chart(results_cg)


Prompt for index 0:
    [{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role':
    'user', 'content': ' product 1: POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza, product 2: Pokemon S&S2:
    Rebel Clash BD'}]



### Random smaple V2

In [68]:
folder = "data/results/gpt4all/new_random_sampel"
files = get_json_files(folder)

results_cg = calculate_results(files, folder, model_name="GPT4All")
create_chart(results_cg)


Prompt for index 0:
    Do the following two product decriptions match. Your answer must include yes or no. product 1: Maxxis Maxxis Minion DHR2
    29 x 2.3 Folding Bead, product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Folding Bead)



### Medium Difficulty

In [69]:
folder = "data/results/gpt4all/curated"
files = get_json_files(folder)

results_cg = calculate_results(files, folder, model_name="GPT4All")
create_chart(results_cg)


Prompt for index 0:
    Do the following two product decriptions match. Answer with yes or no. Otherwise I can understand you. product 1: Pen
    Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive

Prompt for index 1:
    You are a helpful assistant that tries to tell if two products are the same. Two products are only the same if all
    attributes match! Only answer yes or no. product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk
    64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive

Prompt for index 2:
    Do the following two product decriptions match. Your answer must include yes or no. product 1: Pen Drive SanDisk 64GB
    Cruzer Glide USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive

Prompt for index 3:
    Are these two products the same? product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk 64GB
    SDIX30N-064G-GN6NN USB 3.0 Flash Drive



### Other runs

In [70]:
folder = "data/results/gpt4all"
files = get_json_files(folder)

results_cg = calculate_results(files, folder, model_name="GPT4All")
create_chart(results_cg)


Prompt for index 0:
    Are these two products the same? product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer
    Men's Special Edition Heuer Monaco Watch

Prompt for index 1:
    Are these two products the same? product 1: Brother HL-L6200DW Wireless High Speed Mono Laser 2-sided printing 250 sheet
    Network 46PPM, product 2: Brother HL-L6300DW Laser Printer - Monochrome - Duplex



## Analyze Aleph Alpha

In [71]:
folder = "data/results/aleph_alpha"
files = get_json_files(folder)

results_cg = calculate_results(files, folder, model_name="Aleph Alpha")
create_chart(results_cg)



Prompt for index 0:
    Are these two products the same? product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer
    Men's Special Edition Heuer Monaco Watch



## Comparing the 3 Models on hard benchmark

In [74]:
files = [{
    "name": "gpt4all",
    "file": "data/results/gpt4all/2023-04-24 16:27:14.136349_simple_promt.json"
},
    {
    "name": "CHAT-GPT",
    "file": "data/results/chat_gpt/2023-04-24 17:20:03.458940_simple_promt.json"},
    {
    "name": "aleph_alpha",
    "file": "data/results/aleph_alpha/2023-04-24 22:42:34.718717_simple_promt.json"}
]

results = []

for file in files:
    df = pd.read_json(file["file"])
    accuracy = calculate_accuracy(df)
    # get the chatbot_question of the 0th row
    question = df.iloc[0]['chatbot_question']
    precision, recall, f1 = calculate_metrics(df)
    results.append({
        "model": file["name"],
        "question": question,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    })

# plot the results
create_models_chart(results, "Comparison of Models on hard benchmark")

Prompt for index 0:
    Are these two products the same? product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer
    Men's Special Edition Heuer Monaco Watch

Prompt for index 1:
    [{'role': 'system', 'content': 'Are these two products the same?'}, {'role': 'user', 'content': "product 1: TAG Heuer
    Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer Monaco Watch"}]

Prompt for index 2:
    Are these two products the same? product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer
    Men's Special Edition Heuer Monaco Watch



## Model comparison on random sample

In [78]:
files = [{
    "name": "gpt4all",
    "file": "data/results/gpt4all/new_random_sampel/2023-05-08 16:09:45.331542_simple_promt.json"
},
    {
    "name": "CHAT-GPT",
    "file": "data/results/chat_gpt/new_random_sample/2023-05-08 11:18:53.559359_simple_promt.json"},
    {
    "name": "aleph_alpha",
    "file": "data/results/aleph_alpha/random_sample_v2/2023-05-09 10:03:42.833719_simple_promt.json"}
]

results = []

for file in files:
    df = pd.read_json(file["file"])
    accuracy = calculate_accuracy(df)
    # get the chatbot_question of the 0th row
    question = df.iloc[0]['chatbot_question']
    precision, recall, f1 = calculate_metrics(df)
    results.append({
        "model": file["name"],
        "question": question,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    })

# plot the results
create_models_chart(results, "Comparison of Models on random benchmark")

Prompt for index 0:
    Do the following two product decriptions match. Your answer must include yes or no. product 1: Maxxis Maxxis Minion DHR2
    29 x 2.3 Folding Bead, product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Folding Bead)

Prompt for index 1:
    [{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role':
    'user', 'content': ' product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead, product 2: Maxxis Minion DHR II 3C
    MaxxTerra/DD TR 29\\" Tire - 29 x 2.3\\" (Folding Bead)'}]

Prompt for index 2:
    Do the following two product decriptions match. Answer with yes or no. product 1: Maxxis Maxxis Minion DHR2 29 x 2.3
    Folding Bead, product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Folding Bead)

